In [1]:
# Import libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import resample

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve
import shap


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier



import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load this file into a Python Dataframe

df = pd.read_csv('2.Preprocessing_Combined_data_5_Years_github_JPAD-2025.csv')
df.tail()

,PATID,Smoking_Status,SBP below 120,SBP 120-140,SBP above 140,DBP below 80,DBP 80-90,DBP above 90,Age_Grp_50-60,Age_Grp_60-70,...,Heart_Disease,Sleep_Apnea,Insomnia,Kidney_Disease,Cholesterol,Vitamin_D_Deficiency,Enlarge_Prostate,Bone_Disease,Depressive_Disorder,Target
123730,2633300,5,0,0,1,0,0,1,0,1,...,1,0,0,0,0,0,0,0,1,0
123731,1575030,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
123732,2940185,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
123733,1210750,0,0,0,1,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
123734,2430126,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [1]:
df.info()

In [4]:
# Checking counts of 'Target' column
df['Target'].value_counts()
print(df['Target'].value_counts())

Target
0    119723
1      4012
Name: count, dtype: int64


# Drop the 'PATID' column as not relavant to our analysis

In [5]:
df.drop(columns=['PATID', 'EcType_ED', 'EcType_IP', 'EcType_AV'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123735 entries, 0 to 123734
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype
---  ------                    --------------   -----
 0   Smoking_Status            123735 non-null  int64
 1   SBP below 120             123735 non-null  int64
 2   SBP 120-140               123735 non-null  int64
 3   SBP above 140             123735 non-null  int64
 4   DBP below 80              123735 non-null  int64
 5   DBP 80-90                 123735 non-null  int64
 6   DBP above 90              123735 non-null  int64
 7   Age_Grp_50-60             123735 non-null  int64
 8   Age_Grp_60-70             123735 non-null  int64
 9   Age_Grp_70-80             123735 non-null  int64
 10  Age_Grp_80-90             123735 non-null  int64
 11  Age_Grp_90-100            123735 non-null  int64
 12  Sex_F                     123735 non-null  int64
 13  Sex_M                     123735 non-null  int64
 14  AMERICAN_IND/ALASKAN

# Splitting data into feature and target

In [2]:
# Predicted variable is 'Target'
X = df.drop("Target", axis=1)
print(X)
y = df["Target"]
print(y)

In [8]:
# List of age group columns
age_groups = [
    'Age_Grp_90-100', 'Age_Grp_80-90', 'Age_Grp_70-80',
    'Age_Grp_60-70', 'Age_Grp_50-60'
]

# Print initial counts for all age groups
for age_group in age_groups:
    initial_count = df[age_group].value_counts(dropna=False)
    print(f"Initial value counts for '{age_group}':")
    print(initial_count)


Initial value counts for 'Age_Grp_90-100':
Age_Grp_90-100
0    108650
1     15085
Name: count, dtype: int64
Initial value counts for 'Age_Grp_80-90':
Age_Grp_80-90
0    95195
1    28540
Name: count, dtype: int64
Initial value counts for 'Age_Grp_70-80':
Age_Grp_70-80
0    83931
1    39804
Name: count, dtype: int64
Initial value counts for 'Age_Grp_60-70':
Age_Grp_60-70
0    86962
1    36773
Name: count, dtype: int64
Initial value counts for 'Age_Grp_50-60':
Age_Grp_50-60
0    120202
1      3533
Name: count, dtype: int64


# Split data into training and testing set

In [9]:
# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# describes info about train and test set
print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions X_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)

(98988, 49) (24747, 49) (98988,) (24747,)
Number transactions X_train dataset:  (98988, 49)
Number transactions y_train dataset:  (98988,)
Number transactions X_test dataset:  (24747, 49)
Number transactions y_test dataset:  (24747,)


In [10]:
# Check count of Target variables
unique, counts = np.unique(y_test, return_counts=True)
test_class_counts = dict(zip(unique, counts))

print("Class counts in y_test:", test_class_counts)

Class counts in y_test: {0: 23917, 1: 830}


# 1. Logistic Regression Classifier (LR) Model

In [3]:

# Define the parameter grid for Grid Search - Logistic Regression (LR)

param_grid_lr = {
    'logreg__C': [100], 
    'logreg__class_weight': ['balanced'],  
    'logreg__fit_intercept': [True],  
    'logreg__max_iter': [200],  
    'logreg__penalty': ['l2'], 
    'logreg__solver': ['lbfgs']
}



# Define the model and the pipeline
logreg = LogisticRegression(random_state=42)
pipeline_lr = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', logreg)
])

# Inner cross-validation loop for hyperparameter tuning
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define the grid search
grid_search_lr = GridSearchCV(
    estimator=pipeline_lr, 
    param_grid=param_grid_lr, 
    scoring='roc_auc', 
    cv=inner_cv, 
    verbose=2, 
    n_jobs=4
)

# Outer cross-validation loop for model evaluation
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Lists to store the metrics for each fold
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
roc_auc_scores = []
specificity_scores = []
conf_matrices = []
best_params_list = []  

# Nested cross-validation for Logistic Regression
for train_index, test_index in outer_cv.split(X_train, y_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[test_index]
    
    # Grid search on the inner cross-validation loop
    grid_search_lr.fit(X_train_fold, y_train_fold)
    
    # Best estimator
    best_model_lr = grid_search_lr.best_estimator_

    # Best parameters
    best_params_list.append(grid_search_lr.best_params_)
    print("Best Parameters:", best_params_list)

    # Predict on the validation set
    y_probs = best_model_lr.predict_proba(X_val_fold)[:, 1]
    
    # Threshold to Classify
    threshold = 0.5
    y_pred = (y_probs > threshold).astype(int)
    
    # Calculate metrics
    accuracy_scores.append(accuracy_score(y_val_fold, y_pred))
    precision_scores.append(precision_score(y_val_fold, y_pred))
    recall_scores.append(recall_score(y_val_fold, y_pred))
    f1_scores.append(f1_score(y_val_fold, y_pred))
    roc_auc_scores.append(roc_auc_score(y_val_fold, y_probs))
    
    conf_matrix = confusion_matrix(y_val_fold, y_pred)
    conf_matrices.append(conf_matrix)
    
    TN = conf_matrix[0, 0]
    FP = conf_matrix[0, 1]
    specificity = TN / (TN + FP)
    specificity_scores.append(specificity)

# Print average metrics
print(f"Mean Accuracy: {np.mean(accuracy_scores):.4f}")
print(f"Mean Precision: {np.mean(precision_scores):.4f}")
print(f"Mean Recall: {np.mean(recall_scores):.4f}")
print(f"Mean F1 Score: {np.mean(f1_scores):.4f}")
print(f"Mean ROC AUC: {np.mean(roc_auc_scores):.4f}")
print(f"Mean Specificity: {np.mean(specificity_scores):.4f}")

# Confusion matrices for each fold
for i, cm in enumerate(conf_matrices):
    print(f"Confusion Matrix for Fold {i+1}:\n{cm}")

# Evaluation on the test set with the best model from nested CV
best_model_lr.fit(X_train, y_train)
y_probs_test = best_model_lr.predict_proba(X_test)[:, 1]

# Threshold to Classify
threshold = 0.5
y_pred_test = (y_probs_test > threshold).astype(int)

# Calculate test set metrics
test_accuracy_lr = accuracy_score(y_test, y_pred_test)
test_precision_lr = precision_score(y_test, y_pred_test, average='weighted')
test_recall_lr = recall_score(y_test, y_pred_test, average='weighted')
test_f1_lr = f1_score(y_test, y_pred_test, average='weighted')
test_roc_auc_lr = roc_auc_score(y_test, y_probs_test)
test_conf_matrix_lr = confusion_matrix(y_test, y_pred_test)

# Calculate PPV, NPV, and specificity
TP = test_conf_matrix_lr[1, 1]
TN = test_conf_matrix_lr[0, 0]
FP = test_conf_matrix_lr[0, 1]
FN = test_conf_matrix_lr[1, 0]
PPV = TP / (TP + FP)
NPV = TN / (TN + FN)
specificity = TN / (TN + FP)

# Print test set metrics
print(f"Test Accuracy: {test_accuracy_lr:.4f}")
print(f"Test Precision: {test_precision_lr:.4f}")
print(f"Test Recall: {test_recall_lr:.4f}")
print(f"Test F1 Score: {test_f1_lr:.4f}")
print(f"Test ROC AUC: {test_roc_auc_lr:.4f}")
print(f"Test Confusion Matrix_LR Model:\n{test_conf_matrix_lr}")
print(f"Test Positive Predictive Value (PPV): {PPV:.4f}")
print(f"Test Negative Predictive Value (NPV): {NPV:.4f}")
print(f"Test Specificity: {specificity:.4f}")

# Plot the confusion matrix of the test set
sns.heatmap(test_conf_matrix_lr, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.ylabel('Actual/True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix_LR Model - Test Set')
plt.show()

# Calculate the 95% confidence intervals (CIs) for each evaluation metric using bootstrapping
n_bootstraps = 1000
rng = np.random.RandomState(seed=42)

accuracy_scores_boot = []
precision_scores_boot = []
recall_scores_boot = []
f1_scores_boot = []
roc_auc_scores_boot = []
ppv_scores_boot = []
npv_scores_boot = []
specificity_scores_boot = []

for _ in range(n_bootstraps):
    indices = rng.randint(0, len(y_test), len(y_test))
    if len(np.unique(y_test.iloc[indices])) < 2:
        continue
    
    y_test_boot = y_test.iloc[indices]
    y_pred_test_boot = y_pred_test[indices]
    y_probs_test_boot = y_probs_test[indices]
    
    accuracy_scores_boot.append(accuracy_score(y_test_boot, y_pred_test_boot))
    precision_scores_boot.append(precision_score(y_test_boot, y_pred_test_boot))
    recall_scores_boot.append(recall_score(y_test_boot, y_pred_test_boot))
    f1_scores_boot.append(f1_score(y_test_boot, y_pred_test_boot))
    roc_auc_scores_boot.append(roc_auc_score(y_test_boot, y_probs_test_boot))
    
    cm_boot = confusion_matrix(y_test_boot, y_pred_test_boot)
    TP_boot = cm_boot[1, 1]
    TN_boot = cm_boot[0, 0]
    FP_boot = cm_boot[0, 1]
    FN_boot = cm_boot[1, 0]
    ppv_scores_boot.append(TP_boot / (TP_boot + FP_boot) if (TP_boot + FP_boot) > 0 else 0)
    npv_scores_boot.append(TN_boot / (TN_boot + FN_boot) if (TN_boot + FN_boot) > 0 else 0)
    specificity_scores_boot.append(TN_boot / (TN_boot + FP_boot) if (TN_boot + FP_boot) > 0 else 0)

def bootstrap_confidence_interval(data, alpha=0.05):
    sorted_data = np.sort(data)
    lower_bound = np.percentile(sorted_data, 100 * (alpha / 2))
    upper_bound = np.percentile(sorted_data, 100 * (1 - alpha / 2))
    return lower_bound, upper_bound

# Calculate mean values
mean_accuracy = np.mean(accuracy_scores_boot)
mean_precision = np.mean(precision_scores_boot)
mean_recall = np.mean(recall_scores_boot)
mean_f1 = np.mean(f1_scores_boot)
mean_roc_auc = np.mean(roc_auc_scores_boot)
mean_ppv = np.mean(ppv_scores_boot)
mean_npv = np.mean(npv_scores_boot)
mean_specificity = np.mean(specificity_scores_boot)

# Print confidence intervals
print(f"95% CI for Accuracy: {bootstrap_confidence_interval(accuracy_scores_boot)}")
print(f"95% CI for Precision: {bootstrap_confidence_interval(precision_scores_boot)}")
print(f"95% CI for Recall: {bootstrap_confidence_interval(recall_scores_boot)}")
print(f"95% CI for F1 Score: {bootstrap_confidence_interval(f1_scores_boot)}")
print(f"95% CI for ROC AUC: {bootstrap_confidence_interval(roc_auc_scores_boot)}")
print(f"95% CI for PPV: {bootstrap_confidence_interval(ppv_scores_boot)}")
print(f"95% CI for NPV: {bootstrap_confidence_interval(npv_scores_boot)}")
print(f"95% CI for Specificity: {bootstrap_confidence_interval(specificity_scores_boot)}")

# Plot the AUC-ROC curve
fpr, tpr, _ = roc_curve(y_test, y_probs_test)
plt.figure()
plt.plot(fpr, tpr, label=f'AUC-ROC (Mean ROC AUC = {mean_roc_auc:.2f})', color='blue')
plt.plot([0, 1], [0, 1], linestyle='--', color='grey')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('AUC-ROC Curve Logistic Regression Model')
plt.legend(loc="lower right")
plt.show()


# 2. Gradient Boosting Tree Classifier (GBT) Model

In [4]:


# # Define the parameter grid for Grid Search - Gradient Boosting Tree (GBT)
param_grid_gbt = {
    'gbt__n_estimators': [500],
    'gbt__learning_rate': [0.1],
    'gbt__max_depth': [4],
    'gbt__min_samples_split': [5],
    'gbt__min_samples_leaf': [4],
    'gbt__subsample': [1.0],
    'gbt__max_features': ['sqrt']
}


# Define the model and the pipeline
gbt = GradientBoostingClassifier(random_state=42)
pipeline_gbt = Pipeline([
    ('scaler', StandardScaler()),
    ('gbt', gbt)
])

# Inner cross-validation loop for hyperparameter tuning
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define the grid search
grid_search_gbt = GridSearchCV(
    estimator=pipeline_gbt, 
    param_grid=param_grid_gbt, 
    scoring='roc_auc', 
    cv=inner_cv, 
    verbose=2, 
    n_jobs=4
)

# Outer cross-validation loop for model evaluation
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Lists to store the metrics for each fold
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
roc_auc_scores = []
specificity_scores = []
conf_matrices = []
best_params_list = []  

# Nested cross-validation for GBT
for train_index, test_index in outer_cv.split(X_train, y_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[test_index]
    
    # Grid search on the inner cross-validation loop
    grid_search_gbt.fit(X_train_fold, y_train_fold)
    
    # Best estimator
    best_model_gbt = grid_search_gbt.best_estimator_
    
    # Best parameters
    best_params_list.append(grid_search_gbt.best_params_)
    print("Best Parameters:", best_params_list)
    
    # Predict on the validation set
    y_probs = best_model_gbt.predict_proba(X_val_fold)[:, 1]

    # Threshold to Classify
    threshold = 0.5
    y_pred = (y_probs > threshold).astype(int)

    # Calculate metrics
    accuracy_scores.append(accuracy_score(y_val_fold, y_pred))
    precision_scores.append(precision_score(y_val_fold, y_pred))
    recall_scores.append(recall_score(y_val_fold, y_pred))
    f1_scores.append(f1_score(y_val_fold, y_pred))
    roc_auc_scores.append(roc_auc_score(y_val_fold, y_probs))
    conf_matrices.append(confusion_matrix(y_val_fold, y_pred))
    
    # Calculate specificity for the current fold
    TN = conf_matrices[-1][0, 0]
    FP = conf_matrices[-1][0, 1]
    specificity = TN / (TN + FP)
    specificity_scores.append(specificity)

# Print average metrics
print(f"Mean Accuracy: {np.mean(accuracy_scores):.4f}")
print(f"Mean Precision: {np.mean(precision_scores):.4f}")
print(f"Mean Recall: {np.mean(recall_scores):.4f}")
print(f"Mean F1 Score: {np.mean(f1_scores):.4f}")
print(f"Mean ROC AUC: {np.mean(roc_auc_scores):.4f}")
print(f"Mean Specificity: {np.mean(specificity_scores):.4f}")

# Confusion matrices for each fold
for i, cm in enumerate(conf_matrices):
    print(f"Confusion Matrix for Fold {i+1}:\n{cm}")

# Evaluation on the test set with the best model from nested CV
best_model_gbt.fit(X_train, y_train)
y_probs_test = best_model_gbt.predict_proba(X_test)[:, 1]

# Threshold to Classify
threshold = 0.5
y_pred_test = (y_probs_test > threshold).astype(int)

# Calculate test set metrics
test_accuracy_gbt = accuracy_score(y_test, y_pred_test)
test_precision_gbt = precision_score(y_test, y_pred_test, average='weighted')
test_recall_gbt = recall_score(y_test, y_pred_test, average='weighted')
test_f1_gbt = f1_score(y_test, y_pred_test, average='weighted')
test_roc_auc_gbt = roc_auc_score(y_test, y_probs_test)
test_conf_matrix_gbt = confusion_matrix(y_test, y_pred_test)

# Calculate PPV, NPV, and specificity
TP = test_conf_matrix_gbt[1, 1]
TN = test_conf_matrix_gbt[0, 0]
FP = test_conf_matrix_gbt[0, 1]
FN = test_conf_matrix_gbt[1, 0]
PPV = TP / (TP + FP)
NPV = TN / (TN + FN)
specificity = TN / (TN + FP)

# Print test set metrics
print(f"Test Accuracy: {test_accuracy_gbt:.4f}")
print(f"Test Precision: {test_precision_gbt:.4f}")
print(f"Test Recall: {test_recall_gbt:.4f}")
print(f"Test F1 Score: {test_f1_gbt:.4f}")
print(f"Test ROC AUC: {test_roc_auc_gbt:.4f}")
print(f"Test Confusion Matrix_GBT Model:\n{test_conf_matrix_gbt}")
print(f"Test Positive Predictive Value (PPV): {PPV:.4f}")
print(f"Test Negative Predictive Value (NPV): {NPV:.4f}")
print(f"Test Specificity: {specificity:.4f}")

# Plot the confusion matrix of the test set
sns.heatmap(test_conf_matrix_gbt, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.ylabel('Actual/True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix_GBT Model - Test Set')
plt.show()

# Calculate the 95% confidence intervals (CIs) for each evaluation metric using bootstrapping
n_bootstraps = 1000
rng = np.random.RandomState(seed=42)

accuracy_scores_boot = []
precision_scores_boot = []
recall_scores_boot = []
f1_scores_boot = []
roc_auc_scores_boot = []
ppv_scores_boot = []
npv_scores_boot = []
specificity_scores_boot = []

for _ in range(n_bootstraps):
    indices = rng.randint(0, len(y_test), len(y_test))
    if len(np.unique(y_test.iloc[indices])) < 2:
        continue
    
    y_test_boot = y_test.iloc[indices]
    y_pred_test_boot = y_pred_test[indices]
    y_probs_test_boot = y_probs_test[indices]
    
    accuracy_scores_boot.append(accuracy_score(y_test_boot, y_pred_test_boot))
    precision_scores_boot.append(precision_score(y_test_boot, y_pred_test_boot))
    recall_scores_boot.append(recall_score(y_test_boot, y_pred_test_boot))
    f1_scores_boot.append(f1_score(y_test_boot, y_pred_test_boot))
    roc_auc_scores_boot.append(roc_auc_score(y_test_boot, y_probs_test_boot))
    
    cm_boot = confusion_matrix(y_test_boot, y_pred_test_boot)
    TP_boot = cm_boot[1, 1]
    TN_boot = cm_boot[0, 0]
    FP_boot = cm_boot[0, 1]
    FN_boot = cm_boot[1, 0]
    ppv_scores_boot.append(TP_boot / (TP_boot + FP_boot) if (TP_boot + FP_boot) > 0 else 0)
    npv_scores_boot.append(TN_boot / (TN_boot + FN_boot) if (TN_boot + FN_boot) > 0 else 0)
    specificity_scores_boot.append(TN_boot / (TN_boot + FP_boot) if (TN_boot + FP_boot) > 0 else 0)

def bootstrap_confidence_interval(data, alpha=0.05):
    sorted_data = np.sort(data)
    lower_bound = np.percentile(sorted_data, 100 * (alpha / 2))
    upper_bound = np.percentile(sorted_data, 100 * (1 - alpha / 2))
    return lower_bound, upper_bound

# Calculate mean values
mean_accuracy = np.mean(accuracy_scores_boot)
mean_precision = np.mean(precision_scores_boot)
mean_recall = np.mean(recall_scores_boot)
mean_f1 = np.mean(f1_scores_boot)
mean_roc_auc = np.mean(roc_auc_scores_boot)
mean_ppv = np.mean(ppv_scores_boot)
mean_npv = np.mean(npv_scores_boot)
mean_specificity = np.mean(specificity_scores_boot)

# Print confidence intervals
print(f"95% CI for Accuracy: {bootstrap_confidence_interval(accuracy_scores_boot)}")
print(f"95% CI for Precision: {bootstrap_confidence_interval(precision_scores_boot)}")
print(f"95% CI for Recall: {bootstrap_confidence_interval(recall_scores_boot)}")
print(f"95% CI for F1 Score: {bootstrap_confidence_interval(f1_scores_boot)}")
print(f"95% CI for ROC AUC: {bootstrap_confidence_interval(roc_auc_scores_boot)}")
print(f"95% CI for PPV: {bootstrap_confidence_interval(ppv_scores_boot)}")
print(f"95% CI for NPV: {bootstrap_confidence_interval(npv_scores_boot)}")
print(f"95% CI for Specificity: {bootstrap_confidence_interval(specificity_scores_boot)}")

# Plot the AUC-ROC curve
fpr, tpr, _ = roc_curve(y_test, y_probs_test)
plt.figure()
plt.plot(fpr, tpr, label=f'AUC-ROC (Mean ROC AUC = {mean_roc_auc:.2f})', color='blue')
plt.plot([0, 1], [0, 1], linestyle='--', color='grey')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('AUC-ROC Curve GBT Model')
plt.legend(loc="lower right")
plt.show()

# SHapley Additive exPlanations (SHAP) Analysis

In [5]:

# Initialize a SHAP explainer with the best model and the training dataset
explainer = shap.Explainer(best_model_gbt.named_steps['gbt'], X_train)

# Compute SHAP values for the test dataset
shap_values = explainer(X_test)

# Top indices based on mean absolute SHAP values
def get_top_shap_indices(shap_values, top_n=15):
    # Sum absolute SHAP values across all samples to get feature importance
    shap_sum = np.abs(shap_values.values).mean(axis=0)
    
    # Get indices of the top features
    top_indices = np.argsort(shap_sum)[-top_n:]
    return top_indices

# Get top 15 feature indices from SHAP values
top_indices = get_top_shap_indices(shap_values, 15)

# Extract SHAP values for the top 15 features
shap_values_top = shap_values[:, top_indices]

if isinstance(X_test, pd.DataFrame):
    X_test_top = X_test.iloc[:, top_indices]
    feature_names = X_test.columns[top_indices]
else:
    X_test_top = X_test[:, top_indices]
    feature_names = top_indices  

# Create SHAP Explanation object
shap_values_explanation = shap.Explanation(
    values=shap_values_top,
    base_values=shap_values.base_values,
    data=X_test_top,
    feature_names=feature_names
)

# Plot with the built-in SHAP function
plt.figure()
shap.summary_plot(shap_values_explanation, X_test_top, max_display=15, plot_type="dot", show=False)
plt.savefig('SHAP_Plot_5Years_GBT_Model.png', bbox_inches='tight')
plt.close()

# Plot the bar plot with SHAP values
plt.figure()
shap.summary_plot(shap_values_explanation, X_test_top, max_display=15, plot_type="bar", show=False)

# Add values on top of bars with two decimal points
shap_sum = np.abs(shap_values.values).mean(axis=0)
top_indices_sorted = np.argsort(shap_sum)[-15:]
sorted_shap_values = shap_sum[top_indices_sorted]
sorted_feature_names = X_test.columns[top_indices_sorted]

for i, (value, name) in enumerate(zip(sorted_shap_values, sorted_feature_names)):
    plt.text(value, i, f'{value:.2f}', va='center')

plt.savefig('SHAP_Plot_5Years_GBT_Model_Bar.png', bbox_inches='tight')
plt.close()  


# SHAP FORCE plot - GBT Model

In [6]:

# Initialize a SHAP explainer with the best model and the training dataset
explainer = shap.Explainer(best_model_gbt.named_steps['gbt'], X_train)

# Compute SHAP values for the test dataset
shap_values = explainer(X_test)

# Function to get the top indices based on mean absolute SHAP values
def get_top_shap_indices(shap_values, top_n=15):
    # Sum absolute SHAP values across all samples to get feature importance
    shap_sum = np.abs(shap_values.values).mean(axis=0)
    
    # Get indices of the top features
    top_indices = np.argsort(shap_sum)[-top_n:]
    return top_indices

# Get top 15 feature indices from SHAP values
top_indices = get_top_shap_indices(shap_values, 15)

# Extract SHAP values for the top 15 features
shap_values_top = shap_values[:, top_indices]

if isinstance(X_test, pd.DataFrame):
    X_test_top = X_test.iloc[:, top_indices]
    feature_names = X_test.columns[top_indices]
else:
    X_test_top = X_test[:, top_indices]
    feature_names = top_indices  

# Create SHAP Explanation object
shap_values_explanation = shap.Explanation(
    values=shap_values_top.values,
    base_values=shap_values.base_values,
    data=X_test_top,
    feature_names=feature_names
)

# Global force plot
shap.initjs()
shap.force_plot(
    shap_values_explanation.base_values.mean(), 
    shap_values_explanation.values.mean(axis=0), 
    feature_names, 
    matplotlib=True
)
plt.savefig('SHAP_Global_5Years_Force_Plot_GBT_Model.png', bbox_inches='tight')
plt.close()  

# Individual force plot for the first instance
shap.initjs()
shap.force_plot(
    shap_values_explanation.base_values[0], 
    shap_values_explanation.values[0], 
    X_test_top.iloc[0], 
    feature_names, 
    matplotlib=True
)
plt.savefig('SHAP_Individual_5Years_Force_Plot_GBT_Model.png', bbox_inches='tight')
plt.close()  


# 3. Light GBM Classifier Model (Light GBM)

In [7]:


# Define the parameter grid for Grid Search - (Light GBM)

param_grid_lgbm = {
    'lgbm__n_estimators': [100, 200],
    'lgbm__learning_rate': [0.01, 0.1],
    'lgbm__max_depth': [5, 7],
    'lgbm__num_leaves': [31, 40],
    'lgbm__subsample': [0.8, 0.9],
    'lgbm__colsample_bytree': [0.8, 1.0]
}

# Define the model and the pipeline
lgbm = lgb.LGBMClassifier(random_state=42)
pipeline_lgbm = Pipeline([
    ('scaler', StandardScaler()),
    ('lgbm', lgbm)
])

# Inner cross-validation loop for hyperparameter tuning
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define the grid search
grid_search_lgbm = GridSearchCV(
    estimator=pipeline_lgbm, 
    param_grid=param_grid_lgbm, 
    scoring='roc_auc', 
    cv=inner_cv, 
    verbose=2, 
    n_jobs=4
)

# Outer cross-validation loop for model evaluation
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Lists to store the metrics for each fold
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
roc_auc_scores = []
specificity_scores = []
conf_matrices = []
best_params_list = []  

# Nested cross-validation for LightGBM
for train_index, test_index in outer_cv.split(X_train, y_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[test_index]
    
    # Grid search on the inner cross-validation loop
    grid_search_lgbm.fit(X_train_fold, y_train_fold)
    
    # Best estimator
    best_model_lgbm = grid_search_lgbm.best_estimator_
    
    # Best parameters
    best_params_list.append(grid_search_lgbm.best_params_)
    print("Best Parameters:", best_params_list)

    # Predict on the validation set
    y_probs = best_model_lgbm.predict_proba(X_val_fold)[:, 1]
    
    # Threshold to Classify
    threshold = 0.5
    y_pred = (y_probs > threshold).astype(int)
    
    # Calculate metrics
    accuracy_scores.append(accuracy_score(y_val_fold, y_pred))
    precision_scores.append(precision_score(y_val_fold, y_pred))
    recall_scores.append(recall_score(y_val_fold, y_pred))
    f1_scores.append(f1_score(y_val_fold, y_pred))
    roc_auc_scores.append(roc_auc_score(y_val_fold, y_probs))
    conf_matrices.append(confusion_matrix(y_val_fold, y_pred))

# Print average metrics
print(f"Mean Accuracy: {np.mean(accuracy_scores):.4f}")
print(f"Mean Precision: {np.mean(precision_scores):.4f}")
print(f"Mean Recall: {np.mean(recall_scores):.4f}")
print(f"Mean F1 Score: {np.mean(f1_scores):.4f}")
print(f"Mean ROC AUC: {np.mean(roc_auc_scores):.4f}")

# Confusion matrices for each fold
for i, cm in enumerate(conf_matrices):
    print(f"Confusion Matrix for Fold {i+1}:\n{cm}")

# Evaluation on the test set with the best model from nested CV
best_model_lgbm.fit(X_train, y_train)
y_probs_test = best_model_lgbm.predict_proba(X_test)[:, 1]

# Threshold to Classify
threshold = 0.5
y_pred_test = (y_probs_test > threshold).astype(int)

# Calculate test set metrics
test_accuracy_lgbm = accuracy_score(y_test, y_pred_test)
test_precision_lgbm = precision_score(y_test, y_pred_test, average='weighted')
test_recall_lgbm = recall_score(y_test, y_pred_test, average='weighted')
test_f1_lgbm = f1_score(y_test, y_pred_test, average='weighted')
test_roc_auc_lgbm = roc_auc_score(y_test, y_probs_test)
test_conf_matrix_lgbm = confusion_matrix(y_test, y_pred_test)

# Calculate PPV, NPV, and specificity
TP = test_conf_matrix_lgbm[1, 1]
TN = test_conf_matrix_lgbm[0, 0]
FP = test_conf_matrix_lgbm[0, 1]
FN = test_conf_matrix_lgbm[1, 0]
PPV = TP / (TP + FP)
NPV = TN / (TN + FN)
specificity = TN / (TN + FP)

# Print test set metrics
print(f"Test Accuracy: {test_accuracy_lgbm:.4f}")
print(f"Test Precision: {test_precision_lgbm:.4f}")
print(f"Test Recall: {test_recall_lgbm:.4f}")
print(f"Test F1 Score: {test_f1_lgbm:.4f}")
print(f"Test ROC AUC: {test_roc_auc_lgbm:.4f}")
print(f"Test Confusion Matrix_LGBM Model:\n{test_conf_matrix_lgbm}")
print(f"Test Positive Predictive Value (PPV): {PPV:.4f}")
print(f"Test Negative Predictive Value (NPV): {NPV:.4f}")
print(f"Test Specificity: {specificity:.4f}")

# Plot the confusion matrix of the test set
sns.heatmap(test_conf_matrix_lgbm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.ylabel('Actual/True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix_LGBM Model - Test Set')
plt.show()

# Calculate the 95% confidence intervals (CIs) for each evaluation metric using bootstrapping
n_bootstraps = 1000
rng = np.random.RandomState(seed=42)

accuracy_scores_boot = []
precision_scores_boot = []
recall_scores_boot = []
f1_scores_boot = []
roc_auc_scores_boot = []
ppv_scores_boot = []
npv_scores_boot = []
specificity_scores_boot = []

for _ in range(n_bootstraps):
    indices = rng.randint(0, len(y_test), len(y_test))
    if len(np.unique(y_test.iloc[indices])) < 2:
        continue
    
    y_test_boot = y_test.iloc[indices]
    y_pred_test_boot = y_pred_test[indices]
    y_probs_test_boot = y_probs_test[indices]
    
    accuracy_scores_boot.append(accuracy_score(y_test_boot, y_pred_test_boot))
    precision_scores_boot.append(precision_score(y_test_boot, y_pred_test_boot))
    recall_scores_boot.append(recall_score(y_test_boot, y_pred_test_boot))
    f1_scores_boot.append(f1_score(y_test_boot, y_pred_test_boot))
    roc_auc_scores_boot.append(roc_auc_score(y_test_boot, y_probs_test_boot))
    
    cm_boot = confusion_matrix(y_test_boot, y_pred_test_boot)
    TP_boot = cm_boot[1, 1]
    TN_boot = cm_boot[0, 0]
    FP_boot = cm_boot[0, 1]
    FN_boot = cm_boot[1, 0]
    ppv_scores_boot.append(TP_boot / (TP_boot + FP_boot) if (TP_boot + FP_boot) > 0 else 0)
    npv_scores_boot.append(TN_boot / (TN_boot + FN_boot) if (TN_boot + FN_boot) > 0 else 0)
    specificity_scores_boot.append(TN_boot / (TN_boot + FP_boot) if (TN_boot + FP_boot) > 0 else 0)

def bootstrap_confidence_interval(data, alpha=0.05):
    sorted_data = np.sort(data)
    lower_bound = np.percentile(sorted_data, 100 * (alpha / 2))
    upper_bound = np.percentile(sorted_data, 100 * (1 - alpha / 2))
    return lower_bound, upper_bound

print(f"95% CI for Accuracy: {bootstrap_confidence_interval(accuracy_scores_boot)}")
print(f"95% CI for Precision: {bootstrap_confidence_interval(precision_scores_boot)}")
print(f"95% CI for Recall: {bootstrap_confidence_interval(recall_scores_boot)}")
print(f"95% CI for F1 Score: {bootstrap_confidence_interval(f1_scores_boot)}")
print(f"95% CI for ROC AUC: {bootstrap_confidence_interval(roc_auc_scores_boot)}")
print(f"95% CI for PPV: {bootstrap_confidence_interval(ppv_scores_boot)}")
print(f"95% CI for NPV: {bootstrap_confidence_interval(npv_scores_boot)}")
print(f"95% CI for Specificity: {bootstrap_confidence_interval(specificity_scores_boot)}")

# Calculate mean values
mean_accuracy = np.mean(accuracy_scores_boot)
mean_precision = np.mean(precision_scores_boot)
mean_recall = np.mean(recall_scores_boot)
mean_f1 = np.mean(f1_scores_boot)
mean_roc_auc = np.mean(roc_auc_scores_boot)
mean_ppv = np.mean(ppv_scores_boot)
mean_npv = np.mean(npv_scores_boot)
mean_specificity = np.mean(specificity_scores_boot)

# Plot the AUC-ROC curve
fpr, tpr, _ = roc_curve(y_test, y_probs_test)
plt.figure()
plt.plot(fpr, tpr, label=f'AUC-ROC (Mean ROC AUC = {mean_roc_auc:.2f})', color='blue')
plt.plot([0, 1], [0, 1], linestyle='--', color='grey')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('AUC-ROC Curve LightGBM Model')
plt.legend(loc="lower right")
plt.show()


# 4. XGBoost Classifier Model (XGBoost)

In [8]:

# Define the parameter grid for Grid Search - (XGBoost)

param_grid_xgb = {
    'xgb__n_estimators': [200],
    'xgb__learning_rate': [0.05],
    'xgb__max_depth': [5],
    'xgb__min_child_weight': [5],
    'xgb__subsample': [0.8],
    'xgb__colsample_bytree': [0.8]
}

# Define the model and the pipeline
xgb_model = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
pipeline_xgb = Pipeline([
    ('scaler', StandardScaler()),
    ('xgb', xgb_model)
])

# Inner cross-validation loop for hyperparameter tuning
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define the grid search
grid_search_xgb = GridSearchCV(
    estimator=pipeline_xgb, 
    param_grid=param_grid_xgb, 
    scoring='roc_auc', 
    cv=inner_cv, 
    verbose=2, 
    n_jobs=4
)

# Outer cross-validation loop for model evaluation
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Lists to store the metrics for each fold
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
roc_auc_scores = []
conf_matrices = []
best_params_list = [] 

# Nested cross-validation for XGBoost
for train_index, test_index in outer_cv.split(X_train, y_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[test_index]
    
    # Grid search on the inner cross-validation loop
    grid_search_xgb.fit(X_train_fold, y_train_fold)
    
    # Best estimator
    best_model_xgb = grid_search_xgb.best_estimator_
    
    # Best parameters
    best_params_list.append(grid_search_xgb.best_params_)
    print("Best Parameters:", best_params_list)

    # Predict on the validation set
    y_probs = best_model_xgb.predict_proba(X_val_fold)[:, 1]
    
    # Threshold to Classify
    threshold = 0.5
    y_pred = (y_probs > threshold).astype(int)
    
    # Calculate metrics
    accuracy_scores.append(accuracy_score(y_val_fold, y_pred))
    precision_scores.append(precision_score(y_val_fold, y_pred))
    recall_scores.append(recall_score(y_val_fold, y_pred))
    f1_scores.append(f1_score(y_val_fold, y_pred))
    roc_auc_scores.append(roc_auc_score(y_val_fold, y_probs))
    conf_matrices.append(confusion_matrix(y_val_fold, y_pred))

# Print average metrics
print(f"Mean Accuracy: {np.mean(accuracy_scores):.4f}")
print(f"Mean Precision: {np.mean(precision_scores):.4f}")
print(f"Mean Recall: {np.mean(recall_scores):.4f}")
print(f"Mean F1 Score: {np.mean(f1_scores):.4f}")
print(f"Mean ROC AUC: {np.mean(roc_auc_scores):.4f}")

# Print confusion matrices for each fold
for i, cm in enumerate(conf_matrices):
    print(f"Confusion Matrix for Fold {i+1}:\n{cm}")

# Evaluation on the test set with the best model from nested CV
best_model_xgb.fit(X_train, y_train)
y_probs_test = best_model_xgb.predict_proba(X_test)[:, 1]

# Threshold to Classify
threshold = 0.5
y_pred_test = (y_probs_test > threshold).astype(int)

# Calculate test set metrics
test_accuracy_xgb = accuracy_score(y_test, y_pred_test)
test_precision_xgb = precision_score(y_test, y_pred_test, average='weighted')
test_recall_xgb = recall_score(y_test, y_pred_test, average='weighted')
test_f1_xgb = f1_score(y_test, y_pred_test, average='weighted')
test_roc_auc_xgb = roc_auc_score(y_test, y_probs_test)
test_conf_matrix_xgb = confusion_matrix(y_test, y_pred_test)

# Calculate PPV, NPV, and specificity
TP = test_conf_matrix_xgb[1, 1]
TN = test_conf_matrix_xgb[0, 0]
FP = test_conf_matrix_xgb[0, 1]
FN = test_conf_matrix_xgb[1, 0]
PPV = TP / (TP + FP)
NPV = TN / (TN + FN)
specificity = TN / (TN + FP)

# Print test set metrics
print(f"Test Accuracy: {test_accuracy_xgb:.4f}")
print(f"Test Precision: {test_precision_xgb:.4f}")
print(f"Test Recall: {test_recall_xgb:.4f}")
print(f"Test F1 Score: {test_f1_xgb:.4f}")
print(f"Test ROC AUC: {test_roc_auc_xgb:.4f}")
print(f"Test Confusion Matrix_XGB Model:\n{test_conf_matrix_xgb}")
print(f"Test Positive Predictive Value (PPV): {PPV:.4f}")
print(f"Test Negative Predictive Value (NPV): {NPV:.4f}")
print(f"Test Specificity: {specificity:.4f}")

# Plot the confusion matrix of the test set
sns.heatmap(test_conf_matrix_xgb, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.ylabel('Actual/True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix_XGB Model - Test Set')
plt.show()

# Calculate the 95% confidence intervals (CIs) for each evaluation metric using bootstrapping
n_bootstraps = 1000
rng = np.random.RandomState(seed=42)

accuracy_scores_boot = []
precision_scores_boot = []
recall_scores_boot = []
f1_scores_boot = []
roc_auc_scores_boot = []
ppv_scores_boot = []
npv_scores_boot = []
specificity_scores_boot = []

for _ in range(n_bootstraps):
    indices = rng.randint(0, len(y_test), len(y_test))
    if len(np.unique(y_test.iloc[indices])) < 2:
        continue
    
    y_test_boot = y_test.iloc[indices]
    y_pred_test_boot = y_pred_test[indices]
    y_probs_test_boot = y_probs_test[indices]
    
    accuracy_scores_boot.append(accuracy_score(y_test_boot, y_pred_test_boot))
    precision_scores_boot.append(precision_score(y_test_boot, y_pred_test_boot))
    recall_scores_boot.append(recall_score(y_test_boot, y_pred_test_boot))
    f1_scores_boot.append(f1_score(y_test_boot, y_pred_test_boot))
    roc_auc_scores_boot.append(roc_auc_score(y_test_boot, y_probs_test_boot))
    
    cm_boot = confusion_matrix(y_test_boot, y_pred_test_boot)
    TP_boot = cm_boot[1, 1]
    TN_boot = cm_boot[0, 0]
    FP_boot = cm_boot[0, 1]
    FN_boot = cm_boot[1, 0]
    ppv_scores_boot.append(TP_boot / (TP_boot + FP_boot) if (TP_boot + FP_boot) > 0 else 0)
    npv_scores_boot.append(TN_boot / (TN_boot + FN_boot) if (TN_boot + FN_boot) > 0 else 0)
    specificity_scores_boot.append(TN_boot / (TN_boot + FP_boot) if (TN_boot + FP_boot) > 0 else 0)

def bootstrap_confidence_interval(data, alpha=0.05):
    sorted_data = np.sort(data)
    lower_bound = np.percentile(sorted_data, 100 * (alpha / 2))
    upper_bound = np.percentile(sorted_data, 100 * (1 - alpha / 2))
    return lower_bound, upper_bound

print(f"95% CI for Accuracy: {bootstrap_confidence_interval(accuracy_scores_boot)}")
print(f"95% CI for Precision: {bootstrap_confidence_interval(precision_scores_boot)}")
print(f"95% CI for Recall: {bootstrap_confidence_interval(recall_scores_boot)}")
print(f"95% CI for F1 Score: {bootstrap_confidence_interval(f1_scores_boot)}")
print(f"95% CI for ROC AUC: {bootstrap_confidence_interval(roc_auc_scores_boot)}")
print(f"95% CI for PPV: {bootstrap_confidence_interval(ppv_scores_boot)}")
print(f"95% CI for NPV: {bootstrap_confidence_interval(npv_scores_boot)}")
print(f"95% CI for Specificity: {bootstrap_confidence_interval(specificity_scores_boot)}")

# Calculate mean values
mean_accuracy = np.mean(accuracy_scores_boot)
mean_precision = np.mean(precision_scores_boot)
mean_recall = np.mean(recall_scores_boot)
mean_f1 = np.mean(f1_scores_boot)
mean_roc_auc = np.mean(roc_auc_scores_boot)
mean_ppv = np.mean(ppv_scores_boot)
mean_npv = np.mean(npv_scores_boot)
mean_specificity = np.mean(specificity_scores_boot)

# Plot the AUC-ROC curve
fpr, tpr, _ = roc_curve(y_test, y_probs_test)
plt.figure()
plt.plot(fpr, tpr, label=f'AUC-ROC (Mean ROC AUC = {mean_roc_auc:.2f})', color='blue')
plt.plot([0, 1], [0, 1], linestyle='--', color='grey')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('AUC-ROC Curve XGBoost Model')
plt.legend(loc="lower right")
plt.show()


# 5. Random Forest Classifier Model (RF)

In [9]:

# Define the parameter grid for Grid Search - Random Forest (RF)
param_grid_rf = {
    'rf__n_estimators': [200],
    'rf__max_depth': [30],
    'rf__min_samples_split': [8],
    'rf__min_samples_leaf': [5],
    'rf__bootstrap': [True]
}

# Define the model and the pipeline
rf_model = RandomForestClassifier(random_state=42)
pipeline_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', rf_model)
])

# Inner cross-validation loop for hyperparameter tuning
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define the grid search
grid_search_rf = GridSearchCV(
    estimator=pipeline_rf, 
    param_grid=param_grid_rf, 
    scoring='roc_auc', 
    cv=inner_cv, 
    verbose=2, 
    n_jobs=4
)

# Outer cross-validation loop for model evaluation
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Lists to store the metrics for each fold
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
roc_auc_scores = []
specificity_scores = []
conf_matrices = []
best_params_list = []  

# Nested cross-validation for Random Forest
for train_index, test_index in outer_cv.split(X_train, y_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[test_index]
    
    # Grid search on the inner cross-validation loop
    grid_search_rf.fit(X_train_fold, y_train_fold)
    
    # Best estimator
    best_model_rf = grid_search_rf.best_estimator_
    
    # Best parameters
    best_params_list.append(grid_search_rf.best_params_)
    print("Best Parameters:", best_params_list)

    # Predict on the validation set
    y_probs = best_model_rf.predict_proba(X_val_fold)[:, 1]
    
    # Threshold to Classify
    threshold = 0.5
    y_pred = (y_probs > threshold).astype(int)
    
    # Calculate metrics
    accuracy_scores.append(accuracy_score(y_val_fold, y_pred))
    precision_scores.append(precision_score(y_val_fold, y_pred))
    recall_scores.append(recall_score(y_val_fold, y_pred))
    f1_scores.append(f1_score(y_val_fold, y_pred))
    roc_auc_scores.append(roc_auc_score(y_val_fold, y_probs))
    
    conf_matrix = confusion_matrix(y_val_fold, y_pred)
    conf_matrices.append(conf_matrix)
    
    TN = conf_matrix[0, 0]
    FP = conf_matrix[0, 1]
    specificity = TN / (TN + FP)
    specificity_scores.append(specificity)

# Print average metrics
print(f"Mean Accuracy: {np.mean(accuracy_scores):.4f}")
print(f"Mean Precision: {np.mean(precision_scores):.4f}")
print(f"Mean Recall: {np.mean(recall_scores):.4f}")
print(f"Mean F1 Score: {np.mean(f1_scores):.4f}")
print(f"Mean ROC AUC: {np.mean(roc_auc_scores):.4f}")
print(f"Mean Specificity: {np.mean(specificity_scores):.4f}")

# Confusion matrices for each fold
for i, cm in enumerate(conf_matrices):
    print(f"Confusion Matrix for Fold {i+1}:\n{cm}")

# Evaluation on the test set with the best model from nested CV
best_model_rf.fit(X_train, y_train)
y_probs_test = best_model_rf.predict_proba(X_test)[:, 1]

# Threshold to Classify
threshold = 0.5
y_pred_test = (y_probs_test > threshold).astype(int)

# Calculate test set metrics
test_accuracy_rf = accuracy_score(y_test, y_pred_test)
test_precision_rf = precision_score(y_test, y_pred_test, average='weighted')
test_recall_rf = recall_score(y_test, y_pred_test, average='weighted')
test_f1_rf = f1_score(y_test, y_pred_test, average='weighted')
test_roc_auc_rf = roc_auc_score(y_test, y_probs_test)
test_conf_matrix_rf = confusion_matrix(y_test, y_pred_test)

# Calculate PPV, NPV, and specificity
TP = test_conf_matrix_rf[1, 1]
TN = test_conf_matrix_rf[0, 0]
FP = test_conf_matrix_rf[0, 1]
FN = test_conf_matrix_rf[1, 0]
PPV = TP / (TP + FP)
NPV = TN / (TN + FN)
specificity = TN / (TN + FP)

# Print test set metrics
print(f"Test Accuracy: {test_accuracy_rf:.4f}")
print(f"Test Precision: {test_precision_rf:.4f}")
print(f"Test Recall: {test_recall_rf:.4f}")
print(f"Test F1 Score: {test_f1_rf:.4f}")
print(f"Test ROC AUC: {test_roc_auc_rf:.4f}")
print(f"Test Confusion Matrix_RF Model:\n{test_conf_matrix_rf}")
print(f"Test Positive Predictive Value (PPV): {PPV:.4f}")
print(f"Test Negative Predictive Value (NPV): {NPV:.4f}")
print(f"Test Specificity: {specificity:.4f}")

# Plot the confusion matrix of the test set
sns.heatmap(test_conf_matrix_rf, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.ylabel('Actual/True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix_RF Model - Test Set')
plt.show()

# Calculate the 95% confidence intervals (CIs) for each evaluation metric using bootstrapping
n_bootstraps = 1000
rng = np.random.RandomState(seed=42)

accuracy_scores_boot = []
precision_scores_boot = []
recall_scores_boot = []
f1_scores_boot = []
roc_auc_scores_boot = []
ppv_scores_boot = []
npv_scores_boot = []
specificity_scores_boot = []

for _ in range(n_bootstraps):
    indices = rng.randint(0, len(y_test), len(y_test))
    if len(np.unique(y_test.iloc[indices])) < 2:
        continue
    
    y_test_boot = y_test.iloc[indices]
    y_pred_test_boot = y_pred_test[indices]
    y_probs_test_boot = y_probs_test[indices]
    
    accuracy_scores_boot.append(accuracy_score(y_test_boot, y_pred_test_boot))
    precision_scores_boot.append(precision_score(y_test_boot, y_pred_test_boot))
    recall_scores_boot.append(recall_score(y_test_boot, y_pred_test_boot))
    f1_scores_boot.append(f1_score(y_test_boot, y_pred_test_boot))
    roc_auc_scores_boot.append(roc_auc_score(y_test_boot, y_probs_test_boot))
    
    cm_boot = confusion_matrix(y_test_boot, y_pred_test_boot)
    TP_boot = cm_boot[1, 1]
    TN_boot = cm_boot[0, 0]
    FP_boot = cm_boot[0, 1]
    FN_boot = cm_boot[1, 0]
    ppv_scores_boot.append(TP_boot / (TP_boot + FP_boot) if (TP_boot + FP_boot) > 0 else 0)
    npv_scores_boot.append(TN_boot / (TN_boot + FN_boot) if (TN_boot + FN_boot) > 0 else 0)
    specificity_scores_boot.append(TN_boot / (TN_boot + FP_boot) if (TN_boot + FP_boot) > 0 else 0)

def bootstrap_confidence_interval(data, alpha=0.05):
    sorted_data = np.sort(data)
    lower_bound = np.percentile(sorted_data, 100 * (alpha / 2))
    upper_bound = np.percentile(sorted_data, 100 * (1 - alpha / 2))
    return lower_bound, upper_bound

# Calculate mean values
mean_accuracy = np.mean(accuracy_scores_boot)
mean_precision = np.mean(precision_scores_boot)
mean_recall = np.mean(recall_scores_boot)
mean_f1 = np.mean(f1_scores_boot)
mean_roc_auc = np.mean(roc_auc_scores_boot)
mean_ppv = np.mean(ppv_scores_boot)
mean_npv = np.mean(npv_scores_boot)
mean_specificity = np.mean(specificity_scores_boot)

# Print confidence intervals
print(f"95% CI for Accuracy: {bootstrap_confidence_interval(accuracy_scores_boot)}")
print(f"95% CI for Precision: {bootstrap_confidence_interval(precision_scores_boot)}")
print(f"95% CI for Recall: {bootstrap_confidence_interval(recall_scores_boot)}")
print(f"95% CI for F1 Score: {bootstrap_confidence_interval(f1_scores_boot)}")
print(f"95% CI for ROC AUC: {bootstrap_confidence_interval(roc_auc_scores_boot)}")
print(f"95% CI for PPV: {bootstrap_confidence_interval(ppv_scores_boot)}")
print(f"95% CI for NPV: {bootstrap_confidence_interval(npv_scores_boot)}")
print(f"95% CI for Specificity: {bootstrap_confidence_interval(specificity_scores_boot)}")

# Plot the AUC-ROC curve
fpr, tpr, _ = roc_curve(y_test, y_probs_test)
plt.figure()
plt.plot(fpr, tpr, label=f'AUC-ROC (Mean ROC AUC = {mean_roc_auc:.2f})', color='blue')
plt.plot([0, 1], [0, 1], linestyle='--', color='grey')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('AUC-ROC Curve Random Forest Model')
plt.legend(loc="lower right")
plt.show()


# 6. AdaBoost Classifier Model (AdaBoost)

In [10]:

# Define the parameter grid - AdaBoost
param_grid_ada = {
    'ada__n_estimators': [200],
    'ada__learning_rate': [0.1],
    'ada__estimator__max_depth': [3]
}


# Define the model and the pipeline
ada = AdaBoostClassifier(estimator=DecisionTreeClassifier(random_state=42), random_state=42)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('ada', ada)
])


# Inner cross-validation loop for hyperparameter tuning
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define the grid search
grid_search = GridSearchCV(
    estimator=pipeline, 
    param_grid=param_grid_ada, 
    scoring='roc_auc', 
    cv=inner_cv, 
    verbose=2, 
    n_jobs=4
)

# Outer cross-validation loop for model evaluation
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Lists to store the metrics for each fold
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
roc_auc_scores = []
specificity_scores = []
conf_matrices = []
best_params_list = []

# Nested cross-validation for AdaBoost
for train_index, test_index in outer_cv.split(X_train, y_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[test_index]
    
    # Grid search on the inner cross-validation loop
    grid_search.fit(X_train_fold, y_train_fold)
    
    # Best estimator
    best_model = grid_search.best_estimator_
    
    # Best parameters
    best_params_list.append(grid_search.best_params_)
    print("Best Parameters:", best_params_list)

    # Predict on the validation set
    y_probs = best_model.predict_proba(X_val_fold)[:, 1]
    
    # Threshold to Classify
    threshold = 0.5
    y_pred = (y_probs > threshold).astype(int)
    
    # Calculate metrics
    accuracy_scores.append(accuracy_score(y_val_fold, y_pred))
    precision_scores.append(precision_score(y_val_fold, y_pred))
    recall_scores.append(recall_score(y_val_fold, y_pred))
    f1_scores.append(f1_score(y_val_fold, y_pred))
    roc_auc_scores.append(roc_auc_score(y_val_fold, y_probs))
    
    conf_matrix = confusion_matrix(y_val_fold, y_pred)
    conf_matrices.append(conf_matrix)
    
    TN = conf_matrix[0, 0]
    FP = conf_matrix[0, 1]
    specificity = TN / (TN + FP)
    specificity_scores.append(specificity)

# Print average metrics
print(f"Mean Accuracy: {np.mean(accuracy_scores):.4f}")
print(f"Mean Precision: {np.mean(precision_scores):.4f}")
print(f"Mean Recall: {np.mean(recall_scores):.4f}")
print(f"Mean F1 Score: {np.mean(f1_scores):.4f}")
print(f"Mean ROC AUC: {np.mean(roc_auc_scores):.4f}")
print(f"Mean Specificity: {np.mean(specificity_scores):.4f}")

# Confusion matrices for each fold
for i, cm in enumerate(conf_matrices):
    print(f"Confusion Matrix for Fold {i+1}:\n{cm}")

# Evaluation on the test set with the best model from nested CV
best_model.fit(X_train, y_train)
y_probs_test = best_model.predict_proba(X_test)[:, 1]

# Use a Threshold to Classify
threshold = 0.5
y_pred_test = (y_probs_test > threshold).astype(int)

# Calculate test set metrics
test_accuracy_ada = accuracy_score(y_test, y_pred_test)
test_precision_ada = precision_score(y_test, y_pred_test, average='weighted')
test_recall_ada = recall_score(y_test, y_pred_test, average='weighted')
test_f1_ada = f1_score(y_test, y_pred_test, average='weighted')
test_roc_auc_ada = roc_auc_score(y_test, y_probs_test)
test_conf_matrix_ada = confusion_matrix(y_test, y_pred_test)

# Calculate PPV, NPV, and specificity
TP = test_conf_matrix_ada[1, 1]
TN = test_conf_matrix_ada[0, 0]
FP = test_conf_matrix_ada[0, 1]
FN = test_conf_matrix_ada[1, 0]
PPV = TP / (TP + FP)
NPV = TN / (TN + FN)
specificity = TN / (TN + FP)

# Print test set metrics
print(f"Test Accuracy: {test_accuracy_ada:.4f}")
print(f"Test Precision: {test_precision_ada:.4f}")
print(f"Test Recall: {test_recall_ada:.4f}")
print(f"Test F1 Score: {test_f1_ada:.4f}")
print(f"Test ROC AUC: {test_roc_auc_ada:.4f}")
print(f"Test Confusion Matrix_AdaBoost Model:\n{test_conf_matrix_ada}")
print(f"Test Positive Predictive Value (PPV): {PPV:.4f}")
print(f"Test Negative Predictive Value (NPV): {NPV:.4f}")
print(f"Test Specificity: {specificity:.4f}")

# Plot the confusion matrix of the test set
sns.heatmap(test_conf_matrix_ada, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.ylabel('Actual/True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix_AdaBoost Model - Test Set')
plt.show()

# Calculate the 95% confidence intervals (CIs) for each evaluation metric using bootstrapping
n_bootstraps = 1000
rng = np.random.RandomState(seed=42)

accuracy_scores_boot = []
precision_scores_boot = []
recall_scores_boot = []
f1_scores_boot = []
roc_auc_scores_boot = []
ppv_scores_boot = []
npv_scores_boot = []
specificity_scores_boot = []

for _ in range(n_bootstraps):
    indices = rng.randint(0, len(y_test), len(y_test))
    if len(np.unique(y_test.iloc[indices])) < 2:
        continue
    
    y_test_boot = y_test.iloc[indices]
    y_pred_test_boot = y_pred_test[indices]
    y_probs_test_boot = y_probs_test[indices]
    
    accuracy_scores_boot.append(accuracy_score(y_test_boot, y_pred_test_boot))
    precision_scores_boot.append(precision_score(y_test_boot, y_pred_test_boot))
    recall_scores_boot.append(recall_score(y_test_boot, y_pred_test_boot))
    f1_scores_boot.append(f1_score(y_test_boot, y_pred_test_boot))
    roc_auc_scores_boot.append(roc_auc_score(y_test_boot, y_probs_test_boot))
    
    cm_boot = confusion_matrix(y_test_boot, y_pred_test_boot)
    TP_boot = cm_boot[1, 1]
    TN_boot = cm_boot[0, 0]
    FP_boot = cm_boot[0, 1]
    FN_boot = cm_boot[1, 0]
    ppv_scores_boot.append(TP_boot / (TP_boot + FP_boot) if (TP_boot + FP_boot) > 0 else 0)
    npv_scores_boot.append(TN_boot / (TN_boot + FN_boot) if (TN_boot + FN_boot) > 0 else 0)
    specificity_scores_boot.append(TN_boot / (TN_boot + FP_boot) if (TN_boot + FP_boot) > 0 else 0)

def bootstrap_confidence_interval(data, alpha=0.05):
    sorted_data = np.sort(data)
    lower_bound = np.percentile(sorted_data, 100 * (alpha / 2))
    upper_bound = np.percentile(sorted_data, 100 * (1 - alpha / 2))
    return lower_bound, upper_bound

# Calculate mean values
mean_accuracy = np.mean(accuracy_scores_boot)
mean_precision = np.mean(precision_scores_boot)
mean_recall = np.mean(recall_scores_boot)
mean_f1 = np.mean(f1_scores_boot)
mean_roc_auc = np.mean(roc_auc_scores_boot)
mean_ppv = np.mean(ppv_scores_boot)
mean_npv = np.mean(npv_scores_boot)
mean_specificity = np.mean(specificity_scores_boot)

# Print confidence intervals
print(f"95% CI for Accuracy: {bootstrap_confidence_interval(accuracy_scores_boot)}")
print(f"95% CI for Precision: {bootstrap_confidence_interval(precision_scores_boot)}")
print(f"95% CI for Recall: {bootstrap_confidence_interval(recall_scores_boot)}")
print(f"95% CI for F1 Score: {bootstrap_confidence_interval(f1_scores_boot)}")
print(f"95% CI for ROC AUC: {bootstrap_confidence_interval(roc_auc_scores_boot)}")
print(f"95% CI for PPV: {bootstrap_confidence_interval(ppv_scores_boot)}")
print(f"95% CI for NPV: {bootstrap_confidence_interval(npv_scores_boot)}")
print(f"95% CI for Specificity: {bootstrap_confidence_interval(specificity_scores_boot)}")

# Plot the AUC-ROC curve
fpr, tpr, _ = roc_curve(y_test, y_probs_test)
plt.figure()
plt.plot(fpr, tpr, label=f'AUC-ROC (Mean ROC AUC = {mean_roc_auc:.2f})', color='blue')
plt.plot([0, 1], [0, 1], linestyle='--', color='grey')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('AUC-ROC Curve AdaBoost Model')
plt.legend(loc="lower right")
plt.show()


# Combined AUC-ROC Plots

In [11]:

# Function to train a model and get ROC curve data
def train_and_evaluate_model(model, param_grid, X_train, y_train, X_test, y_test):
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('model', model)
    ])
    inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    grid_search = GridSearchCV(
        estimator=pipeline, 
        param_grid=param_grid, 
        scoring='roc_auc', 
        cv=inner_cv, 
        verbose=2, 
        n_jobs=4
    )
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    y_probs_test = best_model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_probs_test)
    return fpr, tpr, roc_auc_score(y_test, y_probs_test), grid_search.best_params_

# Define parameter grids for each model
param_grid_lr = {
    'model__C': [100],
    'model__class_weight': ['balanced'],
    'model__fit_intercept': [True],
    'model__max_iter': [200],
    'model__penalty': ['l2'],
    'model__solver': ['lbfgs']
}
param_grid_gbt = {
    'model__n_estimators': [500],
    'model__learning_rate': [0.1],
    'model__max_depth': [4],
    'model__min_samples_split': [5],
    'model__min_samples_leaf': [4],
    'model__subsample': [1.0],
    'model__max_features': ['sqrt']
}
param_grid_rf = {
    'model__n_estimators': [200],
    'model__max_depth': [30],
    'model__min_samples_split': [8],
    'model__min_samples_leaf': [5],
    'model__bootstrap': [True]
}
param_grid_ada = {
    'model__n_estimators': [200],
    'model__learning_rate': [0.1],
    'model__estimator__max_depth': [3]
}
param_grid_lgbm = {
    'model__n_estimators': [100, 200],
    'model__learning_rate': [0.01, 0.1],
    'model__max_depth': [5, 7],
    'model__num_leaves': [31, 40],
    'model__subsample': [0.8, 0.9],
    'model__colsample_bytree': [0.8, 1.0]
}
param_grid_xgb = {
    'model__n_estimators': [200],
    'model__learning_rate': [0.05],
    'model__max_depth': [5],
    'model__min_child_weight': [5],
    'model__subsample': [0.8],
    'model__colsample_bytree': [0.8]
}

# Train and evaluate each model
fpr_lr, tpr_lr, roc_auc_lr, best_params_lr = train_and_evaluate_model(LogisticRegression(random_state=42), param_grid_lr, X_train, y_train, X_test, y_test)
fpr_gbt, tpr_gbt, roc_auc_gbt, best_params_gbt = train_and_evaluate_model(GradientBoostingClassifier(random_state=42), param_grid_gbt, X_train, y_train, X_test, y_test)
fpr_rf, tpr_rf, roc_auc_rf, best_params_rf = train_and_evaluate_model(RandomForestClassifier(random_state=42), param_grid_rf, X_train, y_train, X_test, y_test)
fpr_ada, tpr_ada, roc_auc_ada, best_params_ada = train_and_evaluate_model(AdaBoostClassifier(estimator=DecisionTreeClassifier(random_state=42), random_state=42), param_grid_ada, X_train, y_train, X_test, y_test)
fpr_lgbm, tpr_lgbm, roc_auc_lgbm, best_params_lgbm = train_and_evaluate_model(lgb.LGBMClassifier(random_state=42), param_grid_lgbm, X_train, y_train, X_test, y_test)
fpr_xgb, tpr_xgb, roc_auc_xgb, best_params_xgb = train_and_evaluate_model(xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'), param_grid_xgb, X_train, y_train, X_test, y_test)

# Plot combined AUC-ROC curves
plt.figure(figsize=(10, 8))
plt.plot(fpr_lr, tpr_lr, label=f'Logistic Regression (AUC = {roc_auc_lr:.2f})')
plt.plot(fpr_gbt, tpr_gbt, label=f'GBT (AUC = {roc_auc_gbt:.2f})')
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC = {roc_auc_rf:.2f})')
plt.plot(fpr_ada, tpr_ada, label=f'AdaBoost (AUC = {roc_auc_ada:.2f})')
plt.plot(fpr_lgbm, tpr_lgbm, label=f'LightGBM (AUC = {roc_auc_lgbm:.2f})')
plt.plot(fpr_xgb, tpr_xgb, label=f'XGBoost (AUC = {roc_auc_xgb:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--', color='black')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")

# Display the plot
plt.show()
